# **Setup**

In [1]:
from utils import Graph, get_dataloader
from utils.interp import *
from utils.dataset.build import generate_planar, generate_misclass 
from models import get_model

from transformer_lens import FactoredMatrix
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
import torch
from torch import Tensor
import einops
import wandb
from jaxtyping import Float
import numpy as np

from tqdm.auto import tqdm
from functools import partial

# **Load Model**

In [ ]:
path = 'trained_models/transformer_2024-09-05_13-28-07'

wandb.init(
    config=path+'.yaml',
    mode='disabled'
)

args = wandb.config

model = get_model(args)
model.eval()
device = model.cfg.device
state_dict = torch.load(path+'.pt',map_location=device)
model.load_state_dict(state_dict)


# **Test model on val set and random graphs**

In [ ]:
loader = get_dataloader('adj','data/n10_15-21_tn_vn/val.npz',1)
n_planar = n_correct = 0
for x,y in loader:
    # plot_graph(adj_to_graph(x.squeeze()))
    logits = model(x)
    correct = logits.argmax()==y
    n_planar += y.item()*correct
    n_correct += correct

print(f"Val Set Accuracy: {(100*n_correct/len(loader)).item():.2f}%")
print(f"{(100*n_planar/n_correct).item():.2f}% of correctly classified graphs were planar.")
print()

for m in range(11,23):
    n_planar = n_correct = 0
    n_samples = 1000
    for _ in range(n_samples):
        g = Graph(nx.gnm_random_graph(args.n_vertices,m))
        y = nx.is_planar(g.G)
        logits = model(g.A)
        correct = logits.argmax()==y
        n_planar += y*correct
        n_correct += correct

        # print(f"y: {y}")
        # print(f"logits.argmax(): {logits.argmax()}")
        # print(f"correct: {correct}")
        # plt.figure(figsize=(8, 6))
        # nx.draw(g, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=10)
        # plt.title("Random Graph")
        # plt.show()

    print(f"Accuracy on {n_samples} random graphs with {m} edges: {(100*n_correct/n_samples).item():.2f}%")
    print(f"{(100*n_planar/n_correct).item():.2f}% of correctly classified graphs were planar.")

# **Cache activations from example graphs**

In [4]:
m = 16

petersen_graph = Graph(nx.petersen_graph())
cycle_graph = Graph(nx.cycle_graph(model.cfg.n_vertices))
complete_graph = Graph(nx.complete_graph(model.cfg.n_vertices))
empty_graph = Graph(nx.empty_graph(model.cfg.n_vertices))
star_graph = Graph(nx.star_graph(model.cfg.n_vertices-1))
random_graph = Graph(nx.gnm_random_graph(model.cfg.n_vertices,m))

loader = get_dataloader('adj','data/n10_15-21_tn_vn/val.npz',128)
batch,batch_labels = next(iter(loader))

model.set_use_attn_result(True)
_, petersen_cache = model.run_with_cache(petersen_graph.A)
_, cycle_cache = model.run_with_cache(cycle_graph.A)
_, complete_cache = model.run_with_cache(complete_graph.A)
_, empty_cache = model.run_with_cache(empty_graph.A)
_, star_cache = model.run_with_cache(star_graph.A)
_, random_cache = model.run_with_cache(random_graph.A)
_, batch_cache = model.run_with_cache(batch)
model.set_use_attn_result(False)

In [ ]:
n_planar = 0
for g in batch:
    graph = Graph(g)
    n_planar += nx.is_planar(graph.G)
print(f"{100*n_planar/len(batch):.2f}% of the batch are planar graphs.")

# **Plot attention patterns**

In [ ]:
random_graph.plot_attention(random_cache["pattern",0])

**Observation**: In Head 1.6 vertices of low degree attend stongly to vertex 2. Let's try and work out how each vertex knows whether or not it has low degree. This can easily be read off from the input but we don't know that's what the model is actually doing. 

My first guess as to how the attention pattern in head 1.6 comes about is this:
- Each vertex of low degree has the concept of "I am a vertex of low degree" encoded in its residual stream position. 
- The query matrix for head 1.6 reads this information from the residual stream and produces queries that encode the concept "I am looking for the special vertex position" in low degree vertex positions.
- The key matrix for head 1.6 produces a key that says "I am in the special vertex position" in position 2. I have no idea why, i guess position 2 might contain useful information that the model wants low degree vertices to have?

If my guess is true, then any degree information would be passed into head 1.6 via Q-composition from components earlier in the model. Let't take a look at how earlier components contribute to the query and key in head 1.6:

In [ ]:
head_index = 6

decomposed_qk_input = decompose_qk_input(model,batch_cache)

decomposed_q = decompose_q(model, decomposed_qk_input, head_index)
decomposed_k = decompose_k(model, decomposed_qk_input, head_index)

component_labels = ["Embed", "PosEmbed"] + [f"0.{h}" for h in range(model.cfg.n_heads)]
for decomposed_input, name in [(decomposed_q, "query"), (decomposed_k, "key")]:
    px.imshow(
        decomposed_input.pow(2).sum(-1).sqrt().mean(0).detach(),
        labels={"x": "Position", "y": "Component"},
        title=f"Norms of components of {name}",
        y=component_labels,
        color_continuous_scale='Blues',
        width=1000, height=400
    ).show()

If we believe that the Norms of the components of the query and the key of head 1.6 are correlated with the importance of that component in the function of head1.6, then these plots support my theory that any degree information is passed into head 1.6 via Q-composition from components earlier in the model.
Indeed if it was passed via K-composition then we would expect to see components of the key which have high norms across all positions, as low degree vertices can occur at any position.

**Further Observations**: 
- We also see that when creating keys, head 1.6 pretty much only cares about the positional information coming from position 2. This suggests that the concept of "I want to gather low degree vertices"&mdash;that is encoded in the position 2 key&mdash;comes entirely from head 1.6 and is not passed on from an earlier component.
- The query plot suggests that, out of the previous components, Embed, PosEmbed and head 0.2, are primarily responsible for passing on degree infomation to the vertex positions in head 1.6, with heads 0.1 and 0.3 playing a less important role.

These plots are not fully convincing though, as the norm is not obviously a principled indicator of causality from earlier components to later ones. We can get a better idea of which components are causaly important for the function of head 1.6 by taking a single example graph and pairing up individual key and query components and seeing what attention scores they produce:

In [ ]:
decomposed_scores = decompose_attn_scores(decomposed_q, decomposed_k)

plot_decomposed_attn_scores(
    decomposed_scores=decomposed_scores,
    cache=batch_cache,
    component_labels=component_labels,
    head_idx=6,
    batch_sample_idx=100
)

These plots make me pretty confident that the QK circuit from head 0.2 to PosEmbed via head 1.6 is responsible for moving information from position 2 to low degree vertices. Specifically, head 0.2 Q-composes with head 1.6 and PosEmbed K-composes with head 1.6.

While I find this all convincing, one problem with the above plots is that they all rely on activations from a single graph input or a small batch input into the model. It would be great if we could show this relationship between components just by looking at the model weights. We do this by studying the *composition scores* of each pair of components:

# **Composition Scores**

In [ ]:
# Get all QK and OV matrices
W_QK = model.W_Q @ model.W_K.transpose(-2, -1)
W_OV = model.W_V @ model.W_O

mlp_0 = model.blocks[0].mlp

# note: adding embed and pos_embed to output spaces may not be very principled (it was my idea and I don't know if anyone else does it)
output_space = [
    (mlp_0(model.W_E) + model.W_E).unsqueeze(dim=0),
    (mlp_0(model.W_pos) + model.W_pos).unsqueeze(dim=0),
    *list(mlp_0(W_OV[0]) + W_OV[0])
]

input_space = {
    'Q': W_QK[1],
    'K': W_QK[1].transpose(-2,-1),
    'V': W_OV[1]
}

# Define tensors to hold the composition scores
composition_scores = {
    "Q": torch.zeros(model.cfg.n_heads+2, model.cfg.n_heads).to(device),
    "K": torch.zeros(model.cfg.n_heads+2, model.cfg.n_heads).to(device),
    "V": torch.zeros(model.cfg.n_heads+2, model.cfg.n_heads).to(device),
}

for i,out_sp in enumerate(output_space):
    for j in range(model.cfg.n_heads):
        for comp_type in "QKV":
            composition_scores[comp_type][i, j] = get_comp_score(out_sp, input_space[comp_type][j])

# baseline
n_samples = 300
comp_scores_baseline = np.zeros(n_samples)
for i in tqdm(range(n_samples)):
    comp_scores_baseline[i] = generate_single_random_comp_score(model)

# px.histogram(
#     comp_scores_baseline,
#     nbins=50,
#     width=800,
#     labels={"x": "Composition score"},
#     title="Random composition scores"
# )

for comp_type in "QKV":
    plot_comp_scores(
        model,
        composition_scores[comp_type], 
        title=f"{comp_type} Composition Scores",
        component_labels=component_labels,
        baseline=comp_scores_baseline.mean()
    )

This strongly supports my theory: it would be a pretty big coincidence if the output space of head 0.2 happend to be the only layer 0 head to have a strong overlap with the query input space of head 1.6 and head 0.2 wasn't integral to head 1.6's functionality. Similarly, we see that the output space of PosEmbed has a strong overlap with the key input space of head 1.6.

If you still aren't convinced then let's take a "corrupted" graph with (in some sense) no low degree vertices as model input and then patch in the cached head 0.2 activations for a "clean" graph with some low degree vertices and see to what extent we can reproduce the "clean" behaviour of head 1.6:

# **Activation Patching**

In [ ]:
heads_to_patch = [[2],[1,3],[1,2,3],[0],[4],[5],[6],[7],[0,4,5,6,7]]

px.imshow(random_cache["pattern",1][0,6],color_continuous_scale='Blues',title="Clean Pattern").show()
px.imshow(petersen_cache["pattern",1][0,6],color_continuous_scale='Blues',title="Corrupted Pattern",zmax=1).show()

model.reset_hooks()
patched_cache = model.add_caching_hooks('blocks.1.attn.hook_pattern')

for heads in heads_to_patch:
    run_with_patched_heads(model,petersen_graph.A,random_cache,heads)

    head_string = ' '.join(f'0.{head}' for head in heads)

    px.imshow(
        patched_cache['blocks.1.attn.hook_pattern'][0,6],
        color_continuous_scale='Blues',
        title=f"Pattern with {head_string} patched",
        zmax=1
    ).show()

In [ ]:
def pattern_metric( 
    patched_cache: ActivationCache,
    clean_cache: ActivationCache,
) -> Float[Tensor, ""]:
    patched_pattern = patched_cache['blocks.1.attn.hook_pattern']
    clean_pattern = clean_cache['blocks.1.attn.hook_pattern']
    return torch.norm(patched_pattern[0,6]-clean_pattern[0,6])

rank_heads_by_patching_metric(model,petersen_graph.A,random_cache,pattern_metric,'blocks.1.attn.hook_pattern')

Now let's follow this trail back through the model by looking more closely at head 0.2.
 
**Observation**: Vertices of high degree attend more to vertices 3 and 6, whereas vertices of low degree seem to attend evenly to all vertices.
Although this pattern has far more evenly distributed&mdash;and therefore smaller&mdash;probabilities, it's functionality seems pretty similar to head 1.6, except with "low degree" swapped with "high degree" and position 2 swapped with positions 3 and 6.

Let's test this theory using similar techniques. As before, we decompose the input to the head into components. Because this is the layer 0 we only have two input components: Embed and PosEmbed:

In [ ]:
decomposed_qk_input_layer_0 = decompose_qk_input(model, batch_cache, layer=0)
decomposed_q_layer_0 = decompose_q(model,decomposed_qk_input_layer_0,head_index=2,layer=0)
decomposed_k_layer_0 = decompose_k(model,decomposed_qk_input_layer_0,head_index=2,layer=0)

component_labels_layer_0 = ["Embed", "PosEmbed"]
for decomposed_input, name in [(decomposed_q_layer_0, "query"), (decomposed_k_layer_0, "key")]:
    px.imshow(
        decomposed_input.pow(2).sum(-1).sqrt().mean(0).detach(),
        labels={"x": "Position", "y": "Component"},
        title=f"Norms of components of {name}",
        y=component_labels_layer_0,
        color_continuous_scale='Blues',
        width=1000, height=400
    ).show()


These plots are very similar to the ones we got for head 1.6, supporting our theory. Indeed, head 0.2 seems like it is only interested in moving information from positions 3 and 6 to vertices based purely on their embedding (and not their position).
To be more certain we will plot the decomposed attention scores:

In [ ]:
decomposed_scores_layer_0 = decompose_attn_scores(decomposed_q_layer_0, decomposed_k_layer_0)

plot_decomposed_attn_scores(
    decomposed_scores=decomposed_scores_layer_0,
    cache=batch_cache,
    component_labels=component_labels_layer_0,
    layer=0,
    head_idx=2,
    batch_sample_idx=100,
    zmax=20
)

Now my questions are: 
- What information is being moved from postions 3 and 6 to the vertices of high degree?
- How exactly does `W_Q[0,2]` give the query vectors a concept of degree based on `input @ W_E`?

# **OV circuit**

In [59]:
def get_OV_circuit(model,layer,head):
    W_O = model.W_O[layer, head]
    W_V = model.W_V[layer, head]
    W_E = model.W_E
    W_U = model.W_U

    OV_circuit = FactoredMatrix(W_V, W_O)
    full_OV_circuit = W_E @ OV_circuit @ W_U
    return full_OV_circuit, OV_circuit

In [ ]:
zmax = 0.4

layers = list(range(model.cfg.n_layers))
heads = list(range(model.cfg.n_heads))
fig = make_subplots(
    rows=2, 
    row_titles=[str(i) for i in layers],
    cols=8,
    column_titles=[str(i) for i in heads]
)

for layer in layers:
    for head_index in heads:
        full_OV_circuit, OV_circuit = get_OV_circuit(model,layer,head_index)
        heatmap = go.Heatmap(z=OV_circuit.AB.detach(),zmin=-zmax,zmax=zmax,colorscale='RdBu')
        fig.add_trace(heatmap, row=layer+1, col=head_index+1)
        fig.update_yaxes(autorange='reversed')

# Update the layout for better visualization
fig.update_layout(height=600, width=2000, showlegend=False)

# Show the plot
fig.show()


In [ ]:
px.imshow(
    get_OV_circuit(model,0,2)[1].AB.detach(),
    color_continuous_scale="RdBu"
).show()

# **Visualise neurons**

In [62]:
def plot_acts(model,cache):
    """
    Plots mean neuron activations for all mlp layers in cache.
    """ 
    assert len(cache['embed'].shape) > 2, "No batch dimension found."
    for i in range(model.cfg.n_layers):
        px.imshow(
            cache["mlp_post",i].mean(0), # mean over batch dimension
            y=[f"0.{h}" for h in range(model.cfg.n_vertices)],
            labels={"x": "Neurons", "y": "Vertices"},
            title=f"Layer {i} Neuron Activations",
            color_continuous_scale="Blues"
        ).show()

In [ ]:
plot_acts(model,batch_cache)

# **Correlation between neuron activation and vertex degree**

In [74]:
def get_degree_act_correlation(
    input: Float[Tensor,"batch n_vertices n_vertices"],
    acts: Float[Tensor,"batch n_vertices d_mlp"]
):
    degrees = input.sum(dim=-1) # batch n_vertices
    degrees_flattened = einops.rearrange(
        degrees,
        "batch n_vertices -> 1 (batch n_vertices)"
    )

    acts_flattened = einops.rearrange(
        acts,
        "batch n_vertices d_mlp-> d_mlp (batch n_vertices)"
    )

    degree_acts_stack = torch.cat([degrees_flattened,acts_flattened]) # (batch n_vertices) d_mlp+1
    print(degree_acts_stack.shape)
    correlations = torch.corrcoef(degree_acts_stack)[0,1:] # d_mlp

    return correlations

In [ ]:
loader = get_dataloader('adj','data/n10_15-21_tn_vn/val.npz',128)
input, _ = next(iter(loader))
_, cache = model.run_with_cache(input)
degree_act_correlations = get_degree_act_correlation(input, cache['mlp_post',1])
px.line(
    y=degree_act_correlations.detach(),
    labels={"x": "Neuron", "y": "Correlation with degree"},
    title=f"Correlation between neuron activation and degree"
).show()

# **Logit attribution**

In [ ]:
component_results, component_captions = batch_cache.get_full_resid_decomposition(return_labels=True, expand_neurons=False)
layer_results = torch.stack([v for k,v in batch_cache.items() if 'out' in k])
layer_captions = [k for k in cache if 'out' in k]

logit_diff_directions = get_logit_diff_directions(model,batch_labels)

component_logit_attribution = get_logit_attribution(component_results,logit_diff_directions)
layer_logit_attribution = get_logit_attribution(layer_results,logit_diff_directions)

plot_logit_attribution(component_logit_attribution,component_captions)
plot_logit_attribution(layer_logit_attribution,layer_captions)

**Investigate 1_mlp_out further**

In [ ]:
neuron_results = einops.rearrange(
    batch_cache.get_neuron_results(layer=1),
    "batch n_vertices n_neurons d_model -> n_neurons batch n_vertices d_model"
)

neuron_logit_attribution = get_logit_attribution(neuron_results,logit_diff_directions)

plot_logit_attribution(neuron_logit_attribution,[str(i) for i in range(model.cfg.d_mlp)])

# **Make datasets of specific examples**

In [ ]:
dataset_kwargs = {
    'n': model.cfg.n_vertices,
    'size': 1099,
    'start': 15,
    'end': 22
}

planar_graphs = generate_planar(**dataset_kwargs)

non_planar_graphs = generate_planar(**dataset_kwargs, non_planar=True)

In [ ]:
misclassified_graphs, correct_labels = generate_misclass(
    model=model,
    start=15,
    end=17,
    size=4
)

In [ ]:
for adj in misclassified_graphs:
    g = Graph(adj)
    is_planar,cert = nx.check_planarity(g.G, counterexample=True)
    print(is_planar)
    Graph(cert).plot()

**Study average activations across sets of specific examples**

In [ ]:
model.set_use_attn_result(True)
_, planar_cache = model.run_with_cache(planar_graphs)
_, non_planar_cache = model.run_with_cache(non_planar_graphs)
_, misclassified_cache = model.run_with_cache(misclassified_graphs)
model.set_use_attn_result(False)

plot_acts(model,planar_cache)
plot_acts(model,non_planar_cache)
plot_acts(model,misclassified_cache)

**Logit attributions across sets of specific examples**

In [ ]:
planar_neuron_results = einops.rearrange(
    planar_cache.get_neuron_results(layer=1),
    "batch n_vertices n_neurons d_model -> n_neurons batch n_vertices d_model"
)

non_planar_neuron_results = einops.rearrange(
    non_planar_cache.get_neuron_results(layer=1),
    "batch n_vertices n_neurons d_model -> n_neurons batch n_vertices d_model"
)

misclassified_neuron_results = einops.rearrange(
    misclassified_cache.get_neuron_results(layer=1),
    "batch n_vertices n_neurons d_model -> n_neurons batch n_vertices d_model"
)

planar_logit_diff_directions = get_logit_diff_directions(model,torch.ones(dataset_kwargs['size'],dtype=torch.int64))
non_planar_logit_diff_directions = get_logit_diff_directions(model,torch.zeros(dataset_kwargs['size'],dtype=torch.int64))
misclassified_logit_diff_directions = get_logit_diff_directions(model,torch.zeros(len(misclassified_graphs),dtype=torch.int64))

planar_neuron_logit_attribution = get_logit_attribution(planar_neuron_results,planar_logit_diff_directions)
non_planar_neuron_logit_attribution = get_logit_attribution(non_planar_neuron_results,non_planar_logit_diff_directions)
misclassified_neuron_logit_attribution = get_logit_attribution(misclassified_neuron_results,misclassified_logit_diff_directions)

plot_logit_attribution(planar_neuron_logit_attribution,[str(i) for i in range(model.cfg.d_mlp)])
plot_logit_attribution(non_planar_neuron_logit_attribution,[str(i) for i in range(model.cfg.d_mlp)])
plot_logit_attribution(misclassified_neuron_logit_attribution,[str(i) for i in range(model.cfg.d_mlp)])